In [1]:
# import necessary libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from ast import literal_eval
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.datasets import make_gaussian_quantiles
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('hatexplain_feature_extractions_v3.csv',
                 converters={'word_index_glove': literal_eval, 
                              'lstm_feature_map': literal_eval,
                              'tfidf_word1gram': literal_eval,
                              'tfidf_word2gram': literal_eval,
                              'tfidf_word3gram': literal_eval,
                              'w2v_cbow': literal_eval})

In [3]:
df

,Unnamed: 0,vote_ordinal,word_index_glove,lstm_feature_map,tfidf_word1gram,tfidf_word2gram,tfidf_word3gram,w2v_cbow
0,0,1,"[46768, 269, 169, 71222, 298, 298, 10625, 3255...","[0.13938053, -0.18590486, -0.060539033, 0.0901...","[4834, 17394, 6750, 1188, 19043, 19043, 11943,...","[36687, 139476, 52026, 9263, 153473, 153092, 9...","[39463, 155726, 56397, 10094, 173291, 172093, ...","[161, 21, 70, 155, 0, 0, 4001, 988, 1, 1, 1, 1..."
1,1,1,"[1, 580, 11853, 248, 1, 138121, 1476, 15082, 2...","[0.04327799, -0.08438103, -0.060883567, 0.0025...","[3571, 2512, 5937, 14569, 19264, 832, 168, 194...","[27018, 18616, 45642, 119516, 155555, 6181, 15...","[29372, 20476, 48977, 134079, 176031, 6848, 16...","[565, 40, 326, 23, 1, 2026, 1264, 118, 167, 1,..."
2,2,1,"[1, 381085, 42443, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.030954387, 0.040644534, -0.07671745, 0.0194...","[11667, 19441, 11893, 8233, 0, 0, 0, 0, 0, 0, ...","[94118, 157521, 97062, 0, 0, 0, 0, 0, 0, 0, 0,...","[105012, 178166, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 672, 1, 279, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
3,3,3,"[1594, 1, 120621, 156181, 65387, 1122, 10360, ...","[0.004926282, 0.0012388171, -0.0033861813, 0.0...","[1757, 3491, 3743, 2964, 13434, 97, 17994, 174...","[12858, 26378, 29004, 22211, 108082, 598, 1454...","[13838, 28731, 31520, 24268, 121193, 620, 1623...","[454, 987, 2862, 640, 221, 1793, 81, 1, 569, 5..."
4,4,3,"[30619, 1, 3623, 12203, 28254, 802, 3214, 5281...","[0.12814277, -0.02799369, -0.022903804, -0.103...","[1758, 19000, 5128, 2393, 2373, 7844, 12652, 7...","[13509, 152337, 38652, 18022, 17904, 62123, 10...","[14667, 169988, 41588, 19867, 19736, 67803, 11...","[10, 1, 313, 3335, 1, 458, 1, 14, 0, 10, 1, 1,..."
...,...,...,...,...,...,...,...,...
20143,20143,2,"[10360, 1361, 11828, 16, 4027, 780, 21675, 1, ...","[-0.08081896, -0.25950265, 0.078297645, 0.0035...","[17994, 17207, 2640, 14893, 403, 4982, 164, 17...","[145398, 137813, 19819, 121393, 3077, 37458, 1...","[162339, 153834, 21830, 136104, 3273, 40289, 1...","[81, 79, 1, 56, 1763, 457, 1861, 1934, 292, 47..."
20144,20144,2,"[405, 16, 5281, 10468, 1, 38846, 1, 22696, 528...","[0.020776447, -0.17637987, 0.11507965, 0.00168...","[6997, 14893, 7509, 17862, 3533, 17828, 12902,...","[55464, 121497, 59401, 143889, 26734, 143516, ...","[60568, 136229, 64905, 160529, 29083, 160148, ...","[41, 56, 14, 29, 396, 622, 4, 1699, 14, 1, 622..."
20145,20145,1,"[69557, 5027, 7449, 102626, 5027, 20971, 13442...","[0.030287618, -0.091516644, -0.115807004, -0.0...","[10311, 4359, 11291, 18950, 4359, 19629, 6680,...","[83848, 33894, 90844, 151973, 33896, 158813, 5...","[93270, 36609, 101069, 169588, 36611, 179677, ...","[1, 2887, 35, 1, 2887, 3239, 1, 1, 1, 1, 1, 1,..."
20146,20146,3,"[34849, 662, 85, 1, 967, 35690, 28379, 298, 1,...","[-0.01178721, -0.16269265, -0.07008049, 0.0428...","[1132, 10144, 19311, 4288, 932, 6419, 1629, 19...","[8861, 82483, 156365, 33511, 6904, 48894, 1198...","[9682, 91762, 176956, 36243, 7573, 52363, 1292...","[1715, 25, 63, 1608, 43, 3, 1, 0, 357, 82, 1, ..."


In [4]:
df['vote_ordinal'].unique()

array([1, 3, 2, 0], dtype=int64)

In [5]:
df1 = df.copy()
df

,Unnamed: 0,vote_ordinal,word_index_glove,lstm_feature_map,tfidf_word1gram,tfidf_word2gram,tfidf_word3gram,w2v_cbow
0,0,1,"[46768, 269, 169, 71222, 298, 298, 10625, 3255...","[0.13938053, -0.18590486, -0.060539033, 0.0901...","[4834, 17394, 6750, 1188, 19043, 19043, 11943,...","[36687, 139476, 52026, 9263, 153473, 153092, 9...","[39463, 155726, 56397, 10094, 173291, 172093, ...","[161, 21, 70, 155, 0, 0, 4001, 988, 1, 1, 1, 1..."
1,1,1,"[1, 580, 11853, 248, 1, 138121, 1476, 15082, 2...","[0.04327799, -0.08438103, -0.060883567, 0.0025...","[3571, 2512, 5937, 14569, 19264, 832, 168, 194...","[27018, 18616, 45642, 119516, 155555, 6181, 15...","[29372, 20476, 48977, 134079, 176031, 6848, 16...","[565, 40, 326, 23, 1, 2026, 1264, 118, 167, 1,..."
2,2,1,"[1, 381085, 42443, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.030954387, 0.040644534, -0.07671745, 0.0194...","[11667, 19441, 11893, 8233, 0, 0, 0, 0, 0, 0, ...","[94118, 157521, 97062, 0, 0, 0, 0, 0, 0, 0, 0,...","[105012, 178166, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 672, 1, 279, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
3,3,3,"[1594, 1, 120621, 156181, 65387, 1122, 10360, ...","[0.004926282, 0.0012388171, -0.0033861813, 0.0...","[1757, 3491, 3743, 2964, 13434, 97, 17994, 174...","[12858, 26378, 29004, 22211, 108082, 598, 1454...","[13838, 28731, 31520, 24268, 121193, 620, 1623...","[454, 987, 2862, 640, 221, 1793, 81, 1, 569, 5..."
4,4,3,"[30619, 1, 3623, 12203, 28254, 802, 3214, 5281...","[0.12814277, -0.02799369, -0.022903804, -0.103...","[1758, 19000, 5128, 2393, 2373, 7844, 12652, 7...","[13509, 152337, 38652, 18022, 17904, 62123, 10...","[14667, 169988, 41588, 19867, 19736, 67803, 11...","[10, 1, 313, 3335, 1, 458, 1, 14, 0, 10, 1, 1,..."
...,...,...,...,...,...,...,...,...
20143,20143,2,"[10360, 1361, 11828, 16, 4027, 780, 21675, 1, ...","[-0.08081896, -0.25950265, 0.078297645, 0.0035...","[17994, 17207, 2640, 14893, 403, 4982, 164, 17...","[145398, 137813, 19819, 121393, 3077, 37458, 1...","[162339, 153834, 21830, 136104, 3273, 40289, 1...","[81, 79, 1, 56, 1763, 457, 1861, 1934, 292, 47..."
20144,20144,2,"[405, 16, 5281, 10468, 1, 38846, 1, 22696, 528...","[0.020776447, -0.17637987, 0.11507965, 0.00168...","[6997, 14893, 7509, 17862, 3533, 17828, 12902,...","[55464, 121497, 59401, 143889, 26734, 143516, ...","[60568, 136229, 64905, 160529, 29083, 160148, ...","[41, 56, 14, 29, 396, 622, 4, 1699, 14, 1, 622..."
20145,20145,1,"[69557, 5027, 7449, 102626, 5027, 20971, 13442...","[0.030287618, -0.091516644, -0.115807004, -0.0...","[10311, 4359, 11291, 18950, 4359, 19629, 6680,...","[83848, 33894, 90844, 151973, 33896, 158813, 5...","[93270, 36609, 101069, 169588, 36611, 179677, ...","[1, 2887, 35, 1, 2887, 3239, 1, 1, 1, 1, 1, 1,..."
20146,20146,3,"[34849, 662, 85, 1, 967, 35690, 28379, 298, 1,...","[-0.01178721, -0.16269265, -0.07008049, 0.0428...","[1132, 10144, 19311, 4288, 932, 6419, 1629, 19...","[8861, 82483, 156365, 33511, 6904, 48894, 1198...","[9682, 91762, 176956, 36243, 7573, 52363, 1292...","[1715, 25, 63, 1608, 43, 3, 1, 0, 357, 82, 1, ..."


In [6]:
df = df.drop('Unnamed: 0', axis=1)

In [7]:
df['vote_ordinal'].unique()

array([1, 3, 2, 0], dtype=int64)

## 1. ML Models using LSTM

In [8]:
X = df['lstm_feature_map']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

### 1.1 Decision Tree with LSTM features

In [9]:
C1 = DecisionTreeClassifier()

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}
# grid search
grid_search = GridSearchCV(estimator=C1, param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search.fit(X_train, y_train)
grid_search.best_estimator_
dt_best = grid_search.best_estimator_
print('Best Parameters using Decision Tree',dt_best)

"""Training with Decision Tree"""
C1 = dt_best
C1 = C1.fit(X_train, y_train)
y_pred_C1 = C1.predict(X_test)

print("Accuracy of Decision Tree with best parameters:",metrics.accuracy_score(y_test, y_pred_C1).round(3))

#y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred_C1))

Fitting 4 folds for each of 50 candidates, totalling 200 fits
Best Parameters using Decision Tree DecisionTreeClassifier(max_depth=5, min_samples_leaf=5)
Accuracy of Decision Tree with best parameters: 0.434
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       286
           1       0.42      0.87      0.56      2318
           2       0.50      0.10      0.17      1675
           3       0.50      0.25      0.34      1766

    accuracy                           0.43      6045
   macro avg       0.35      0.31      0.27      6045
weighted avg       0.45      0.43      0.36      6045



C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 1.2 AdaBoost with LSTM features

In [10]:
X = df['lstm_feature_map']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train = list(np.array(X_train))
X_test = list(np.array(X_test))

In [11]:
abc = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=300, learning_rate=1)

abc.fit(X_train, y_train)

errors = []

for y_pred in (abc.staged_predict(X_test)):
    errors.append(1.0 - accuracy_score(y_pred, y_test))
    
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       265
           1       0.46      0.58      0.51      2326
           2       0.38      0.30      0.33      1668
           3       0.46      0.45      0.45      1786

    accuracy                           0.44      6045
   macro avg       0.32      0.33      0.32      6045
weighted avg       0.42      0.44      0.42      6045



## 2. ML Model with tfidf_word1gram

In [12]:
X = df['tfidf_word1gram']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

### 2.1 tfidf_word1gram with Decision Tree

In [13]:
X = df['tfidf_word1gram']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [14]:
C1 = DecisionTreeClassifier()

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=C1, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search.fit(X_train, y_train)
grid_search.best_estimator_
dt_best = grid_search.best_estimator_
print('Best Parameters',dt_best)

"""Training with Decision Tree"""
C1 = dt_best
C1 = C1.fit(X_train, y_train)
y_pred_C1 = C1.predict(X_test)

print("Accuracy of Decision Tree with best parameters:",metrics.accuracy_score(y_test, y_pred_C1).round(3))

#y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred_C1))

Fitting 4 folds for each of 50 candidates, totalling 200 fits
Best Parameters DecisionTreeClassifier(max_depth=5, min_samples_leaf=20)
Accuracy of Decision Tree with best parameters: 0.399
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       292
           1       0.42      0.68      0.52      2379
           2       0.30      0.02      0.03      1588
           3       0.37      0.43      0.39      1786

    accuracy                           0.40      6045
   macro avg       0.27      0.28      0.24      6045
weighted avg       0.35      0.40      0.33      6045



C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 2.2 tf-idf word1gram with AdaBoost

In [15]:
X = df['tfidf_word1gram']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [16]:
abc = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=300, learning_rate=1)

abc.fit(X_train, y_train)

errors = []

for y_pred in (abc.staged_predict(X_test)):
    errors.append(1.0 - accuracy_score(y_pred, y_test))
    
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.05      0.02      0.03       294
           1       0.47      0.65      0.54      2358
           2       0.36      0.23      0.28      1588
           3       0.51      0.46      0.48      1805

    accuracy                           0.45      6045
   macro avg       0.35      0.34      0.33      6045
weighted avg       0.43      0.45      0.43      6045



## 3. w2v_cbow

### 3.1 w2v_cbow with Decision Tree

In [17]:
X = df['w2v_cbow']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [18]:
C1 = DecisionTreeClassifier()

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=C1, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search.fit(X_train, y_train)
grid_search.best_estimator_
dt_best = grid_search.best_estimator_
print('Best Parameters',dt_best)

"""Training with Decision Tree"""
C1 = dt_best
C1 = C1.fit(X_train, y_train)
y_pred_C1 = C1.predict(X_test)

print("Accuracy of Decision Tree with best parameters:",metrics.accuracy_score(y_test, y_pred_C1).round(3))

#y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred_C1))

Fitting 4 folds for each of 50 candidates, totalling 200 fits
Best Parameters DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=5)
Accuracy of Decision Tree with best parameters: 0.409
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       284
           1       0.41      0.83      0.55      2377
           2       0.54      0.05      0.10      1627
           3       0.38      0.23      0.29      1757

    accuracy                           0.41      6045
   macro avg       0.33      0.28      0.23      6045
weighted avg       0.42      0.41      0.33      6045



C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 3.2 w2v_cbow with AdaBoost

In [19]:
X = df['w2v_cbow']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [20]:
abc = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=300, learning_rate=1)

abc.fit(X_train, y_train)

errors = []

for y_pred in (abc.staged_predict(X_test)):
    errors.append(1.0 - accuracy_score(y_pred, y_test))
    
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.05      0.01      0.02       288
           1       0.44      0.61      0.51      2314
           2       0.39      0.29      0.34      1650
           3       0.41      0.34      0.37      1793

    accuracy                           0.42      6045
   macro avg       0.32      0.31      0.31      6045
weighted avg       0.40      0.42      0.40      6045



## 4. tfidf_word3gram

### 4.1 tfidf_word3gram with Decision Tree

In [21]:
X = df['tfidf_word3gram']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [22]:
C1 = DecisionTreeClassifier()

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=C1, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search.fit(X_train, y_train)
grid_search.best_estimator_
dt_best = grid_search.best_estimator_

print('Best Parameters',dt_best)

"""Training with Decision Tree"""
C1 = dt_best
C1 = C1.fit(X_train, y_train)
y_pred_C1 = C1.predict(X_test)

print("Accuracy of Decision Tree with best parameters:",metrics.accuracy_score(y_test, y_pred_C1).round(3))

#y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred_C1))

Fitting 4 folds for each of 50 candidates, totalling 200 fits
Best Parameters DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=20)
Accuracy of Decision Tree with best parameters: 0.39
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       280
           1       0.40      0.87      0.54      2358
           2       0.27      0.02      0.03      1617
           3       0.37      0.16      0.22      1790

    accuracy                           0.39      6045
   macro avg       0.26      0.26      0.20      6045
weighted avg       0.33      0.39      0.29      6045



C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 4.2 tfidf_word3gram with AdaBoost

In [23]:
X = df['tfidf_word3gram']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [24]:
abc = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=300, learning_rate=1)

abc.fit(X_train, y_train)

errors = []

for y_pred in (abc.staged_predict(X_test)):
    errors.append(1.0 - accuracy_score(y_pred, y_test))
    
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.06      0.02      0.03       265
           1       0.42      0.64      0.51      2315
           2       0.33      0.18      0.24      1703
           3       0.44      0.38      0.40      1762

    accuracy                           0.41      6045
   macro avg       0.31      0.30      0.30      6045
weighted avg       0.38      0.41      0.38      6045



## 5. word_index_glove

### 5.1 word_index_glove with Decision Tree

In [25]:
X = df['word_index_glove']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [26]:
C1 = DecisionTreeClassifier()

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=C1, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search.fit(X_train, y_train)
grid_search.best_estimator_
dt_best = grid_search.best_estimator_
print('Best Parameters',dt_best)

"""Training with Decision Tree"""
C1 = dt_best
C1 = C1.fit(X_train, y_train)
y_pred_C1 = C1.predict(X_test)

print("Accuracy of Decision Tree with best parameters:",metrics.accuracy_score(y_test, y_pred_C1).round(3))

#y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred_C1))

Fitting 4 folds for each of 50 candidates, totalling 200 fits
Best Parameters DecisionTreeClassifier(max_depth=10, min_samples_leaf=20)
Accuracy of Decision Tree with best parameters: 0.459
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       281
           1       0.45      0.74      0.56      2320
           2       0.40      0.22      0.28      1655
           3       0.54      0.39      0.45      1789

    accuracy                           0.46      6045
   macro avg       0.35      0.34      0.32      6045
weighted avg       0.44      0.46      0.42      6045



C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 5.2 word_index_glove with AdaBoost

In [27]:
X = df['word_index_glove']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [28]:
abc = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=300, learning_rate=1)

abc.fit(X_train, y_train)

errors = []

for y_pred in (abc.staged_predict(X_test)):
    errors.append(1.0 - accuracy_score(y_pred, y_test))
    
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.04      0.01      0.02       283
           1       0.48      0.66      0.56      2300
           2       0.40      0.27      0.32      1666
           3       0.57      0.54      0.55      1796

    accuracy                           0.49      6045
   macro avg       0.37      0.37      0.36      6045
weighted avg       0.47      0.49      0.47      6045



## 6. tfidf_word2gram

### 6.1 tfidf_word3gram with Decision Tree

In [29]:
X = df['word_index_glove']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [30]:
C1 = DecisionTreeClassifier()

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=C1, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

grid_search.fit(X_train, y_train)
grid_search.best_estimator_
dt_best = grid_search.best_estimator_
print('Best Parameters',dt_best)

"""Training with Decision Tree"""
C1 = dt_best
C1 = C1.fit(X_train, y_train)
y_pred_C1 = C1.predict(X_test)

print("Accuracy of Decision Tree with best parameters:",metrics.accuracy_score(y_test, y_pred_C1).round(3))

#y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred_C1))

Fitting 4 folds for each of 50 candidates, totalling 200 fits
Best Parameters DecisionTreeClassifier(max_depth=10, min_samples_leaf=50)
Accuracy of Decision Tree with best parameters: 0.44
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       311
           1       0.44      0.71      0.55      2314
           2       0.37      0.18      0.24      1631
           3       0.47      0.40      0.43      1789

    accuracy                           0.44      6045
   macro avg       0.32      0.32      0.31      6045
weighted avg       0.41      0.44      0.40      6045



C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daisy Adhikari\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

### 5.2 tfidf_word2gram with AdaBoost

In [31]:
X = df['word_index_glove']
y = df.vote_ordinal

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

X_train=list(np.array(X_train))
X_test = list(np.array(X_test))

In [32]:
abc = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2), n_estimators=300, learning_rate=1)

abc.fit(X_train, y_train)

errors = []

for y_pred in (abc.staged_predict(X_test)):
    errors.append(1.0 - accuracy_score(y_pred, y_test))
    
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.04      0.01      0.02       266
           1       0.51      0.68      0.58      2390
           2       0.39      0.30      0.34      1592
           3       0.58      0.49      0.53      1797

    accuracy                           0.50      6045
   macro avg       0.38      0.37      0.37      6045
weighted avg       0.48      0.50      0.48      6045

